# Data Preparation

In [514]:
#Data Preparation
import pandas as pd 
dfList1 = list() # feature all kept




for i in range(1981,2021):
    df = pd.read_csv('NBAcsv/{}.csv'.format(i))
    header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data less the header row
    df.columns = header
    df['season'] = i
    dfList1.append(df)
df = pd.concat(dfList1, axis=0, ignore_index=False) # feature all kept

#remove NA values
is_NaN = df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = df[row_has_NaN]
df = df.fillna(0)
#df.isnull().sum().sum()

#drop player name column for training
player = df['Player']
y=df['Share']
x = df.drop(['Pts Won','Pts Max','Tm','Rank','Player','Share','First'],axis = 1)
df2 = df[['PTS','AST','TRB','WS','WS/48','BLK','G','STL','season','Share']]
x2 = df2.iloc[:,:-1]
index = 474
X_train,X2_train = x[:index],x2[:index]
y_train = y[:index]
X_test,X2_test = x[index:],x2[index:]
y_test = y[index:]
player_test = player[index:]


## Prediction - Linear Regression

In [515]:
def whoismvp(df2,para,returnlist):
    df2 = df2.sort_values(para, ascending=False).drop_duplicates(['season'])
    df2 = df2.sort_values('season',ascending = True)
    df2 = df2[returnlist]
    return df2

In [516]:
from sklearn.model_selection import cross_val_score
import sklearn
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Unselected Feature Data
mvp=whoismvp(df.iloc[index:,:],'Share','Player')
mvp = mvp.to_list()
#Linear Regression 
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)

#score of every player in the test set
returnlist = ['season','pred_score','player','actual_score']
df_pred = X_test
df_pred['pred_score'] = y_pred
df_pred['player'] = player_test
df_pred['actual_score'] = y_test
df_pred = whoismvp(df_pred,'pred_score',returnlist)
df_pred['pred_player'] = df_pred['player']
df_pred = df_pred.drop('player',axis=1)
df_pred['actual_player'] = mvp


# Chosen Feature Data
#Linear Regression 
reg = LinearRegression().fit(X2_train, y_train)
y_pred = reg.predict(X2_test)

#score of every player in the test set
returnlist = ['season','pred_score','player','actual_score']
df_pred2 = X2_test
df_pred2['pred_score'] = y_pred
df_pred2['player'] = player_test
df_pred2['actual_score'] = y_test
df_pred2 = whoismvp(df_pred2,'pred_score',returnlist)
df_pred2['pred_player'] = df_pred2['player']
df_pred2 = df_pred2.drop('player',axis=1)
df_pred2['actual_player'] = mvp

/var/folders/s1/7jctz8h54js5r1x1zgq79dqr0000gn/T/ipykernel_2812/2296639422.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pred['pred_score'] = y_pred
/var/folders/s1/7jctz8h54js5r1x1zgq79dqr0000gn/T/ipykernel_2812/2296639422.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pred['player'] = player_test
/var/folders/s1/7jctz8h54js5r1x1zgq79dqr0000gn/T/ipykernel_2812/2296639422.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [517]:
df_pred

,season,pred_score,actual_score,pred_player,actual_player
5,2008,0.466566,0.159,Chris Paul,Dwyane Wade
1,2009,0.637088,0.98,LeBron James,LeBron James
3,2010,0.422445,0.431,LeBron James,Derrick Rose
1,2011,0.345609,0.888,LeBron James,LeBron James
1,2012,0.537341,0.998,LeBron James,LeBron James
1,2013,0.585251,0.986,Kevin Durant,Kevin Durant
2,2014,0.464972,0.72,James Harden,Stephen Curry
1,2015,0.487571,1,Stephen Curry,Stephen Curry
2,2016,0.568589,0.746,James Harden,Russell Westbrook
1,2017,0.537191,0.955,James Harden,James Harden


# Accuracy- Linear Regression

In [518]:
count = []
for i in range(2008,2021):
    row = df_pred[df_pred['season']==i]
    count.append(row['pred_player']==row['actual_player'])
count = list(map(int,count))
print('Classification Accuracy with Original dataset is '+str(sum(count)/len(count)))

Classification Accuracy with Original dataset is 0.5384615384615384


In [519]:
count = []
for i in range(2008,2021):
    row = df_pred2[df_pred2['season']==i]
    count.append(row['pred_player']==row['actual_player'])
count = list(map(int,count))
print('Classification Accuracy after feature selectionis '+str(sum(count)/len(count)))

Classification Accuracy after feature selectionis 0.5384615384615384


In [520]:
#Regression Accuracy From CV 
from sklearn.model_selection import cross_val_score
score = cross_val_score(reg, x, y, cv=5,scoring='r2').mean()
print('Regression Accuracy with original data is '+str(score))

Regression Accuracy with original data is 0.4391616895019837


In [521]:
#Regression Accuracy From CV 
from sklearn.model_selection import cross_val_score
score = cross_val_score(reg, x2, y, cv=5,scoring='r2').mean()
print('Regression Accuracy after feature selection '+str(score))

Regression Accuracy after feature selection 0.4332792343302697
